In [3]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [4]:
from DatasetManager import DatasetManager

import pandas as pd
import numpy as np

import os
import sys
from sys import argv
import pickle
import random
import joblib

import statistics

import warnings
warnings.filterwarnings('ignore')

In [5]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "prefix"
cls_encoding = "index"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = False
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [6]:
for dataset_name in datasets:

    dataset_manager = DatasetManager(dataset_name)

    for ii in range(n_iter):
        num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))

        for bucket in list(num_buckets):
            bucketID = bucket+1
            print ('Bucket', bucketID)

            #import everything needed to sort and predict
            feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
            cls = joblib.load(cls_path)
            feature_combiner = joblib.load(feat_comb_path)

            #import previously identified samples
            tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
            fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

            sample_instances = []

            with open (tn_path, 'rb') as f:
                tn_list = pickle.load(f)
            with open (tp_path, 'rb') as f:
                tp_list = pickle.load(f)
            with open (fn_path, 'rb') as f:
                fn_list = pickle.load(f)
            with open (fp_path, 'rb') as f:
                fp_list = pickle.load(f)

            #save results to a list
            sample_instances.append(tn_list)
            sample_instances.append(tp_list)
            sample_instances.append(fn_list)
            sample_instances.append(fp_list)
            
            
            for lis in sample_instances:
                for each in lis:
                    dispersal_lime = each['lime_weights_dispersal']
                    dispersal_shap = each['shap_weights_dispersal']
                    adj_dispersal_lime = each['adjusted_lime_weights_dispersal']
                    adj_dispersal_shap = each['adjusted_shap_weights_dispersal']
                    
                    lime_stability = np.mean(dispersal_lime)
                    shap_stability = np.mean(dispersal_shap)
                    adj_lime_stability = np.mean(adj_dispersal_lime)
                    adj_shap_stability = np.mean(adj_dispersal_shap)
                    
                    each['lime_importance_stability'] = lime_stability
                    each['shap_importance_stability'] = shap_stability
                    each['adjusted_lime_importance_stability'] = adj_lime_stability
                    each['adjusted_shap_importance_stability'] = adj_shap_stability

            with open (tn_path, 'wb') as f:
                pickle.dump(sample_instances[0], f)
            with open (tp_path, 'wb') as f:
                pickle.dump(sample_instances[1], f)
            with open (fn_path, 'wb') as f:
                pickle.dump(sample_instances[2], f)
            with open (fp_path, 'wb') as f:
                pickle.dump(sample_instances[3], f)


Bucket 1


KeyError: 'lime_weights_dispersal'

In [7]:
each

{'caseID': '204628',
 'input':         AMOUNT_REQ Case ID    label              Activity Resource  \
 151722     45000.0  204628  deviant  A_SUBMITTED-COMPLETE    112.0   
 
        lifecycle:transition  timesincemidnight  timesincelastevent  \
 151722             COMPLETE              966.0                 0.0   
 
         timesincecasestart  event_nr  month  weekday  hour  open_cases  \
 151722                 0.0       1.0    1.0      5.0  16.0       837.0   
 
             Complete Timestamp  case_length  prefix_nr orig_case_id  
 151722 2012-01-28 16:06:17.749           40          1       204628  ,
 'actual': 1,
 'predicted': 0,
 'proba': 0.5373451411724091,
 'nr_events': 1,
 'pred_type': 'FN',
 'tree_shap_stability': 1.0,
 'shap_weights_dispersal': [5.04873383009714e-16,
  0.0,
  6.111125726756219e-16,
  1.2825063069302494e-15,
  0.0,
  0,
  0,
  0,
  9.914708344218805e-16,
  0,
  0],
 'adjusted_shap_weights_dispersal': [5.04873383009714e-16,
  0.0,
  6.111125726756219e-16,
  1